
# Getting started with pymoo library for multiobjective optimization
---
Based on https://pymoo.org/getting_started/index.html


### Definition
Optimization problem is defined by:
 min $f_m(x)$ for $m = 1,..., M$  &emsp;&emsp;  $\rightarrow$ **$M$ - objective functions to be minimized**
 so that:
 $g_j(x) \leq 0$ for $j = 1,..., K_1$  &emsp;&emsp;&emsp;  $\rightarrow$ **$K_1$ inequality constraints**
 $h_k(x) = 0$ for $k = 1,..., K_2$     &emsp;&emsp;&emsp;  $\rightarrow$ **$K_2$ equality constraints**
 $x_i^L \leq x_i \leq x_i^U$ for $i = 1,..., N$  &emsp;&emsp;  $\rightarrow$ **Lower and Upper bounds**
 and $x \in \Omega$ &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;  $\rightarrow$   **$x$, decision vector from decision space $\Omega$**

